# Import Libraries

Here, we import required libraries. The purpose of each package is as follows.

- glob: to find the data sets
- pandas: to parse csv
- matplotlib: to read ppm file
- cv2: to resize the read images
- os: general operations on path (e.g., os.path.dirname)
- numpy: general numerical operation
- sklearn: to split test data
- tensorflow: CNN

In [40]:
import glob
import pandas as pd
# import matplotlib
import matplotlib.pyplot as plt
# import matplotlib.image as mpimg
import cv2
import os
import numpy as np
from sklearn.model_selection import train_test_split
# from sklearn.metrics import confusion_matrix
import tensorflow as tf
from tensorflow.contrib.layers import flatten
# from pipeline import NewralNetwork, make_adam, Session, build_pipeline

# Read Data for Training

In [66]:
TRAIN_IMAGE_DIR = 'data/GTSRB_Final_Training_Images/GTSRB/Final_Training/Images/'

data_files = []
for train_file in glob.glob(os.path.join(TRAIN_IMAGE_DIR, '*/GT-*.csv')):
    folder = os.path.dirname(train_file)
    df = pd.read_csv(train_file, sep=';')
    df['Filename'] = df['Filename'].apply(lambda x: os.path.join(folder, x))
    data_files.append(df)

In [67]:
train_df = pd.concat(data_files, ignore_index=True)
train_df.head()

,Filename,Width,Height,Roi.X1,Roi.Y1,Roi.X2,Roi.Y2,ClassId
0,data/GTSRB_Final_Training_Images/GTSRB/Final_T...,38,35,5,5,33,30,31
1,data/GTSRB_Final_Training_Images/GTSRB/Final_T...,40,36,6,5,34,31,31
2,data/GTSRB_Final_Training_Images/GTSRB/Final_T...,40,37,5,5,34,32,31
3,data/GTSRB_Final_Training_Images/GTSRB/Final_T...,41,39,6,6,36,34,31
4,data/GTSRB_Final_Training_Images/GTSRB/Final_T...,43,37,6,5,38,32,31


In [84]:
def load_image(image_file):
    """
    Read image file into numpy array (RGB)
    """
    return plt.imread(image_file)

In [145]:
INPUT_SHAPE = (32, 32, 3)

def resize_image(image, shape=INPUT_SHAPE[:2]):
    return cv2.resize(image, shape)

loader = lambda image_file: resize_image(load_image(image_file))
load_all = np.vectorize(loader)

# Load The Data

In [156]:
X = train_df['Filename'].values
y = train_df['ClassId'].values

train_data_filenames, eval_data_filenames, train_labels, eval_labels = train_test_split(X, y, stratify=y, test_size=8000, random_state=0)
train_data = []
for f in train_data_filenames:
    train_data.append(loader(f))
train_data = np.array(train_data, dtype=float)
eval_data = []
for f in eval_data_filenames:
    eval_data.append(loader(f))
eval_data = np.array(eval_data, dtype=float) 
print(len(train_data))
print(len(eval_data))

31209
8000


# Main Function

In [ ]:
gtsrb_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn, model_dir="/tmp/gtsrb_convnet_model")

# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=50)

# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=100,
    num_epochs=None,
    shuffle=True)
gtsrb_classifier.train(
    input_fn=train_input_fn,
    steps=20000,
    hooks=[logging_hook])

  # Evaluate the model and print results
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},
    y=eval_labels,
    num_epochs=1,
    shuffle=False)
eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

if __name__ == "__main__":
  tf.app.run()


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/gtsrb_convnet_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x127e1f518>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/gtsrb_convnet_model/model.ckpt.
INFO:tensorflow

INFO:tensorflow:global_step/sec: 1.25799
INFO:tensorflow:probabilities = [[0.00961209 0.04275379 0.0568032  ... 0.01836062 0.01347492 0.01399498]
 [0.0082774  0.03469462 0.04543413 ... 0.01238074 0.01268274 0.00858661]
 [0.01878978 0.0255833  0.03300976 ... 0.01638716 0.01797376 0.01412671]
 ...
 [0.00596292 0.0508219  0.13322646 ... 0.02446701 0.0029958  0.00246199]
 [0.01817645 0.03458592 0.02957172 ... 0.02183006 0.01362973 0.01314261]
 [0.01761162 0.03072088 0.04280914 ... 0.02221676 0.0151155  0.01443174]] (40.712 sec)
INFO:tensorflow:loss = 3.5823607444763184, step = 701 (79.493 sec)
INFO:tensorflow:probabilities = [[0.01945578 0.03432644 0.02721125 ... 0.01913492 0.01645483 0.02078906]
 [0.01717967 0.03273084 0.0338758  ... 0.02688779 0.01632479 0.02206004]
 [0.02113716 0.02580692 0.02598742 ... 0.02179666 0.01992078 0.0220049 ]
 ...
 [0.01223096 0.02924758 0.04676444 ... 0.01314796 0.00754462 0.01197284]
 [0.00421732 0.03559675 0.12075649 ... 0.01050255 0.01049263 0.01893152]
 

INFO:tensorflow:loss = 2.994582414627075, step = 1401 (68.466 sec)
INFO:tensorflow:probabilities = [[0.01036878 0.01922297 0.02754498 ... 0.00258907 0.00147694 0.00109426]
 [0.00003073 0.00334351 0.02557221 ... 0.00812067 0.00000541 0.00083314]
 [0.00222609 0.04728179 0.0074796  ... 0.01725038 0.00718105 0.00441074]
 ...
 [0.00055006 0.00963961 0.05337203 ... 0.00144965 0.000425   0.00273349]
 [0.000252   0.00033585 0.00161629 ... 0.00039157 0.00000053 0.00012339]
 [0.00062202 0.02131969 0.0064325  ... 0.02042961 0.00031041 0.02526368]] (34.463 sec)
INFO:tensorflow:global_step/sec: 1.40181
INFO:tensorflow:probabilities = [[0.00246184 0.00727831 0.05948115 ... 0.00377686 0.00017572 0.00033985]
 [0.02795985 0.03704755 0.04755696 ... 0.01626081 0.00969761 0.01258713]
 [0.01248812 0.02310235 0.03164857 ... 0.01839068 0.00969613 0.01495441]
 ...
 [0.00007112 0.10397384 0.01230916 ... 0.00000793 0.         0.00000054]
 [0.00055334 0.00812384 0.01079362 ... 0.00457968 0.00018939 0.00009952]
 

# CNN Model

In [159]:
def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  input_layer = tf.reshape(features["x"], [-1] + list(INPUT_SHAPE))

  # Convolutional Layer #1
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #1
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2 and Pooling Layer #2
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Dense Layer
  pool2_flat = tf.reshape(pool2, [-1, 8 * 8 * 64])
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits Layer
  logits = tf.layers.dense(inputs=dropout, units=43)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)